# Reviewing alternative features for options to improve approach

In [ ]:
import os
import re
import datetime
import numpy as np
import pandas as pd
import pandera as pa
import matplotlib.pyplot as plt
import sys
from scipy.stats import pearsonr
from itertools import combinations

import early_hit_ranking as ehr

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
data_dir = '../data'
evolution_filename = os.path.join(data_dir, 'cleaned', 'covid_variants_evolution.parquet')
country_filename = os.path.join(data_dir, 'cleaned', 'country_indicators.parquet')

In [ ]:
evolution_df = pd.read_parquet(evolution_filename)
evolution_locations = evolution_df.location.unique()
len(evolution_locations)

In [ ]:
countries_df = pd.read_parquet(country_filename)

In [ ]:
location_df = pd.read_parquet('../data/processed/location_level_indications.parquet')

### For convenience, identify the columns in country which do not change over time and extract them.

In [ ]:
# The following columns hae only one value per location, which never changes
location_level_columns = (countries_df.groupby('location').nunique() <= 1).all()
location_level_column_names = list(location_level_columns[location_level_columns].index)
location_level_column_names

In [ ]:
# We don't both with test_units as that's tied to the test data, and we add location back in.
columns_we_want_to_keep = [
    'continent', 'location',
    'population_density',
    'median_age', 'aged_65_older', 'aged_70_older',
    'gdp_per_capita', 'extreme_poverty',
    'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers', 'male_smokers', 
    'handwashing_facilities', 'hospital_beds_per_thousand', 'life_expectancy',
    'human_development_index', 'population'
]
location_df  = countries_df[columns_we_want_to_keep].groupby('location').first().reset_index()

## Join the data

In [ ]:
evolution_df.head()

In [ ]:
location_df.head()

In [ ]:
country_continent_mapping = location_df[['continent', 'location']]
combined_df = evolution_df.merge(country_continent_mapping, on='location', how='left')

## Lets plot some stuff

In [ ]:
countries_deaths_df = countries_df.loc[countries_df['location'] == 'World', ['date', 'new_deaths_per_million', 'new_deaths_smoothed_per_million', 'total_deaths_per_million']]

In [ ]:
global_variant_df = evolution_df.groupby(['date', 'variant']).sum().reset_index('date')
global_alpha_df = global_variant_df.loc['Alpha']
global_alpha_df['num_sequences_cumulative'] = global_alpha_df['num_sequences'].cumsum()
global_alpha_df.head()

In [ ]:
global_variant_df = evolution_df[['date', 'num_sequences']].groupby('date').sum().reset_index('date')
global_variant_df['num_sequences_cumulative'] = global_variant_df['num_sequences'].cumsum()
global_variant_df.head()

In [ ]:
plt.plot(global_variant_df['date'], global_variant_df['num_sequences_cumulative'])
plt.plot(countries_deaths_df['date'], countries_deaths_df['total_deaths_per_million'])
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.plot(countries_deaths_df['date'], countries_deaths_df['new_deaths_smoothed_per_million'].cumsum())
plt.plot(countries_deaths_df['date'], countries_deaths_df['total_deaths_per_million'])
plt.legend(['new_deaths_smoothed_per_million', 'total_deaths_per_million'])
plt.show()

In [ ]:
world_df = countries_df.loc[countries_df['location'] == 'World']
fig, ax = plt.subplots()
ax.plot(world_df['date'], world_df['reproduction_rate'])
ax.set_ylabel("reproduction_rate",color="blue",fontsize=14)
ax2=ax.twinx()
# make a plot with different y-axis using second axis object
ax2.plot(world_df['date'], world_df['total_vaccinations_per_hundred'], color='orange')
ax2.set_ylabel("total_vaccinations_per_hundred",color="orange",fontsize=14)
plt.show()


In [ ]:
plt.plot(world_df['date'], world_df['total_vaccinations_per_hundred'])
plt.plot(world_df['date'], world_df['total_vaccinations_per_hundred'])

In [ ]:
world_df = countries_df.loc[countries_df['location'] == 'World']
fig, ax = plt.subplots()
ax.plot(world_df['date'], world_df['excess_mortality_cumulative_per_million'])
ax.set_ylabel("excess_mortality_cumulative_per_million",color="blue",fontsize=14)
ax2=ax.twinx()
# make a plot with different y-axis using second axis object
ax2.plot(world_df['date'], world_df['total_vaccinations_per_hundred'], color='orange')
ax2.set_ylabel("total_vaccinations_per_hundred",color="orange",fontsize=14)
plt.show()


## What happens at the onset of a new variant?

In [ ]:
onsets_df = pd.read_parquet('../data/analysis/location_variant_outbreak.parquet')
onsets_df = onsets_df.set_index(['continent', 'location', 'variant'])
onsets_df.head()

In [ ]:
united_kingdom_onsets_df = onsets_df.loc[('Europe', 'United Kingdom'), :]
united_kingdom_onsets_df

In [ ]:
uk_indicators_df = countries_df.loc[countries_df['location'] == 'United Kingdom', :]
uk_indicators_df.head()

### Do we see a change in reproduction_rate within in an 8 week window around onset of a new variant?

Not really,  or there may be, but it seems dominated by seasonal effects and changes to lockdown policy

In [ ]:
fig, ax = plt.subplots()
leg = []
for variant, row in united_kingdom_onsets_df.iterrows():
    onset_df = uk_indicators_df.loc[
        (uk_indicators_df['date'] > row['outbreak_date'] - pd.to_timedelta(28, unit='d')) & (uk_indicators_df['date'] < row['outbreak_date'] + pd.to_timedelta(28, unit='d'))
    ]
    ax.plot(onset_df['date'], onset_df['reproduction_rate'])
    leg.append(variant)

plt.legend(leg, loc=7, bbox_to_anchor=(1.5, 0.5))
plt.xticks(rotation=90)
plt.ylabel('Reproduction Rate')
plt.suptitle('What happens to the reproduction rate around the outbreak of a new variant?')
plt.title('Shown for United Kingdom; 4 weeks either side of first detected sequence')
plt.show()

In [ ]:
fig, ax = plt.subplots()
leg = []
for variant, row in united_kingdom_onsets_df.iterrows():
    onset_df = uk_indicators_df.loc[
        (uk_indicators_df['date'] > row['outbreak_date'] - pd.to_timedelta(28, unit='d')) & (uk_indicators_df['date'] < row['outbreak_date'] + pd.to_timedelta(28, unit='d'))
    ]
    ax.plot(onset_df['date'], onset_df['new_deaths_smoothed_per_million'])
    leg.append(variant)

plt.legend(leg, loc=7, bbox_to_anchor=(1.5, 0.5))
plt.xticks(rotation=90)
plt.ylabel('new_deaths_smoothed_per_million')
plt.suptitle('What happens to the deaths around the outbreak of a new variant?')
plt.title('Shown for United Kingdom; 4 weeks either side of first detected sequence')
plt.show()

In [ ]:
fig, ax = plt.subplots()
leg = []
for variant, row in united_kingdom_onsets_df.iterrows():
    onset_df = uk_indicators_df.loc[
        (uk_indicators_df['date'] > row['outbreak_date'] - pd.to_timedelta(28, unit='d')) & (uk_indicators_df['date'] < row['outbreak_date'] + pd.to_timedelta(28, unit='d'))
    ]
    ax.plot(onset_df['date'], onset_df['hosp_patients_per_million'])
    leg.append(variant)

plt.legend(leg, loc=7, bbox_to_anchor=(1.5, 0.5))
plt.xticks(rotation=90)
plt.ylabel('hosp_patients_per_million')
#plt.suptitle('What happens to the deaths around the outbreak of a new variant?')
plt.title('Shown for United Kingdom; 4 weeks either side of first detected sequence')
plt.show()

In [ ]:
fig, ax = plt.subplots()
leg = []
for variant, row in united_kingdom_onsets_df.iterrows():
    onset_df = uk_indicators_df.loc[
        (uk_indicators_df['date'] > row['outbreak_date'] - pd.to_timedelta(28, unit='d')) & (uk_indicators_df['date'] < row['outbreak_date'] + pd.to_timedelta(28, unit='d'))
    ]
    ax.plot(onset_df['date'], onset_df['positive_rate'])
    leg.append(variant)

plt.legend(leg, loc=7, bbox_to_anchor=(1.5, 0.5))
plt.xticks(rotation=90)
plt.ylabel('positive_rate')
#plt.suptitle('What happens to the deaths around the outbreak of a new variant?')
plt.title('Shown for United Kingdom; 4 weeks either side of first detected sequence')
plt.show()

In [ ]:
fig, ax = plt.subplots()
leg = []
for variant, row in united_kingdom_onsets_df.iterrows():
    onset_df = uk_indicators_df.loc[
        (uk_indicators_df['date'] > row['outbreak_date'] - pd.to_timedelta(28, unit='d')) & (uk_indicators_df['date'] < row['outbreak_date'] + pd.to_timedelta(28, unit='d'))
    ]
    ax.plot(onset_df['date'], onset_df['stringency_index'])
    leg.append(variant)

plt.legend(leg, loc=7, bbox_to_anchor=(1.5, 0.5))
plt.xticks(rotation=90)
plt.ylabel('stringency_index')
#plt.suptitle('What happens to the deaths around the outbreak of a new variant?')
plt.title('Shown for United Kingdom; 4 weeks either side of first detected sequence')
plt.show()

## Are there any correlations between when a country gets a variant and it's indicators?

In [ ]:
predictors = pd.read_parquet('../data/analysis/predictors.parquet')
predictors.head()

In [ ]:
ranks_df = pd.read_parquet('../data/analysis/location_outbreak.parquet')
ranks_df.head()

In [ ]:
location_df.head()

In [ ]:
location_rank_df = location_df.merge(ranks_df, on='location', how='left')
location_rank_corr_df = location_rank_df.corr()
location_rank_corr_df

In [ ]:
rho = location_rank_df.corr()
pval = location_rank_df.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=t]))
rho.round(2).astype(str) + p

In [ ]:


pvalues = np.zeros(location_rank_corr_df.shape,dtype=np.float64)
col_index = {name: num for num,name in enumerate(location_rank_corr_df.columns)}

for cola, colb in combinations(location_rank_corr_df.columns, 2):
    nanless_locatiion_rank_df = location_rank_df.loc[~location_rank_df[[cola, colb]].isna().any(axis=1)]
    result = pearsonr(nanless_locatiion_rank_df[cola], nanless_locatiion_rank_df[colb])
    cai = col_index[cola]
    cbi = col_index[colb]
    pvalues[cai][cbi] = result.pvalue
    pvalues[cbi][cai] = result.pvalue

pvalues_df = pd.DataFrame(pvalues, columns = location_rank_corr_df.columns, index = location_rank_corr_df.index)
pvalues_df

In [ ]:
result = pearsonr(nanless_locatiion_rank_df[cola], nanless_locatiion_rank_df[colb])

In [ ]:
onsets_df.reset_index(['continent','variant']).head()

In [ ]:
variant_onsets_df = onsets_df.reset_index(['continent','location'])
variant_onsets_df.head()

In [ ]:
from IPython.display import display, Markdown
for variant in variant_onsets_df.index.unique():
    df = location_df.merge(variant_onsets_df.loc[variant, ['location', 'outbreak_rank_global']], on='location', how='left')
    rho = df.corr()
    pval = df.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
    p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=t]))
    display(Markdown('\n#### ' + variant + '\n\n'))
    display(rho.round(2).astype(str) + p)

#### But What about combined global ranking?

In [ ]:
onsets_df.head()

In [ ]:
ranked_df = onsets_df #first_appearance_df.groupby(['continent', 'variant']).rank('min')
ranked_df = ranked_df.reset_index().pivot(index=['continent', 'location'], columns=['variant'], values=['outbreak_date'])
ranked_df.columns = ranked_df.columns.get_level_values(1)
ranked_df

In [ ]:
early_hit_ranked_df = ranked_df.drop(
    ['others', 'non_who'], axis=1
)
early_hit_ranked_df.head()

In [ ]:
early_hit_ranked_df = ehr.calculate_combined_rank(early_hit_ranked_df.transpose())
early_hit_ranked_df.name = 'early_hit_ranking'

In [ ]:
location_rank_df = location_df.merge(early_hit_ranked_df, on='location', how='left')
rho = location_rank_df.corr()
num_comparisons = pow(len(rho.columns),2) - len(rho.columns)
pval = location_rank_df.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=(t/num_comparisons)]))
rho.round(2).astype(str) + p

In [ ]:
pval

In [ ]:
num_comparisons

In [ ]:
pval.loc['population_density', 'population_density']

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
im = ax.imshow(rho)

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(rho.columns)), labels=rho.columns)
ax.set_yticks(np.arange(len(rho.columns)), labels=rho.columns)
# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i, iname in enumerate(rho.columns):
    for j, jname in enumerate(rho.columns):
        text = ax.text(j, i, p.loc[iname, jname],
                       ha="center", va="center", color="w", size=14)
   
plt.colorbar(im, ax=ax)
plt.show()

## So what about adding some mean stats from onset windows?

In [ ]:
onsets_df = pd.read_parquet('../data/analysis/location_variant_outbreak.parquet')
onsets_df = onsets_df.set_index(['continent', 'location', 'variant'])
onsets_df.head()

In [ ]:
per_variant = list()
for variant in onsets_df.index.get_level_values(2).unique():
    #variant = 'Alpha'
    foo = onsets_df.reset_index(['continent', 'location']).loc[variant].merge(countries_df, on=['continent', 'location'], how='left')
    foo = foo.loc[abs(foo['date'] - foo['outbreak_date']) < pd.to_timedelta(14, unit='d'),:]
    foo = foo.groupby(['continent', 'location']).mean(numeric_only=True)
    per_variant.append(foo)
    
foo = pd.concat(per_variant)

#countries_df.loc[countries_df['location'] == 'World', ['date', 'new_deaths_per_million', 'new_deaths_smoothed_per_million', 'total_deaths_per_million']]

In [ ]:
countries_df.columns

In [ ]:
bob = foo[[
    'outbreak_rank_global', 'new_deaths_smoothed_per_million', 'reproduction_rate', 'new_tests_per_thousand', 'total_vaccinations_per_hundred',
    'population_density', 'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate', 'diabetes_prevalence', 
    'female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand', 'life_expectancy', 'human_development_index', 'population'
]]
rho = bob.corr()
num_comparisons = pow(len(rho.columns),2) - len(rho.columns)
pval = bob.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=(t/num_comparisons)]))
rho.round(2).astype(str) + p

fig, ax = plt.subplots(figsize=(12, 12))
im = ax.imshow(rho)

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(rho.columns)), labels=rho.columns)
ax.set_yticks(np.arange(len(rho.columns)), labels=rho.columns)
# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i, iname in enumerate(rho.columns):
    for j, jname in enumerate(rho.columns):
        text = ax.text(j, i, p.loc[iname, jname],
                       ha="center", va="center", color="w", size=14)
   
plt.colorbar(im, ax=ax)
plt.title("Correlation between per variant outbreak order and various country indicators\nNote: lower outbreak_rank_global indicates a country was hit earlier")
plt.show()

In [ ]:
bob = foo[[
    'outbreak_rank_global', 'new_deaths_smoothed_per_million', 'reproduction_rate', 'new_tests_per_thousand', 'total_vaccinations_per_hundred',
    'population_density', 'median_age', 'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate', 'diabetes_prevalence', 
    'female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand', 'life_expectancy', 'human_development_index', 'population'
]]
rho = bob.corr()
num_comparisons = pow(len(rho.columns),2) - len(rho.columns)
pval = bob.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=(t/num_comparisons)]))
rho.round(2).astype(str) + p

fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(rho)

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(rho.columns)), labels=rho.columns)
ax.set_yticks(np.arange(len(rho.columns)), labels=rho.columns)
# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i, iname in enumerate(rho.columns):
    for j, jname in enumerate(rho.columns):
        text = ax.text(j, i, p.loc[iname, jname],
                       ha="center", va="center", color="w", size=14)
   
plt.colorbar(im, ax=ax)
plt.title("Correlation between per variant outbreak order and various country indicators\nNote: lower outbreak_rank_global indicates a country was hit earlier")
plt.show()

### What if we limit the rank calculations to monthly resolution, to account for the fact that reporting isn't even?

In [ ]:
flattened_early_hit_ranked_df = ranked_df.drop(
    ['others', 'non_who'], axis=1
)
flattened_early_hit_ranked_df = flattened_early_hit_ranked_df.applymap(lambda x: pd.to_datetime(x.strftime('%Y-%m-01') if not pd.isnull(x) else pd.NaT))
flattened_early_hit_ranked_df.head()

In [ ]:
flattened_early_hit_ranked_df = ehr.calculate_combined_rank(flattened_early_hit_ranked_df.transpose())
flattened_early_hit_ranked_df.name = 'flattened_early_hit_ranking'

In [ ]:
bob = foo[[
    'outbreak_rank_global', 'new_deaths_smoothed_per_million', 'reproduction_rate', 'new_tests_per_thousand', 'total_vaccinations_per_hundred',
    'population_density', 'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate', 'diabetes_prevalence', 
    'female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand', 'life_expectancy', 'human_development_index', 'population'
]].merge(flattened_early_hit_ranked_df, on='location', how='left')
rho = bob.corr()
num_comparisons = pow(len(rho.columns),2) - len(rho.columns)
pval = bob.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=(t/num_comparisons)]))
rho.round(2).astype(str) + p

fig, ax = plt.subplots(figsize=(12, 12))
im = ax.imshow(rho)

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(rho.columns)), labels=rho.columns)
ax.set_yticks(np.arange(len(rho.columns)), labels=rho.columns)
# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i, iname in enumerate(rho.columns):
    for j, jname in enumerate(rho.columns):
        text = ax.text(j, i, p.loc[iname, jname],
                       ha="center", va="center", color="w", size=14)
   
plt.colorbar(im, ax=ax)
plt.title("Correlation between per variant outbreak order and various country indicators\nNote: lower outbreak_rank_global indicates a country was hit earlier")
plt.show()

### How much difference will that make?

In [ ]:
early_hit_ranked_df = ranked_df.drop(
    ['others', 'non_who'], axis=1
)
#early_hit_ranked_df = early_hit_ranked_df.applymap(lambda x: pd.to_datetime(x.strftime('%Y-%m-01') if not pd.isnull(x) else pd.NaT))
early_hit_ranked_df = ehr.calculate_combined_rank(early_hit_ranked_df.transpose())
early_hit_ranked_df.name = 'early_hit_ranking'
early_hit_ranked_df.head()

In [ ]:
pearsonr(flattened_early_hit_ranked_df, early_hit_ranked_df)

In [ ]:
plt.scatter(flattened_early_hit_ranked_df, early_hit_ranked_df)
plt.ylabel('Schulze Ranking based on actual onset dates')
plt.xlabel('Schulze Ranking based on onset dates by month')

In [ ]:
(flattened_early_hit_ranked_df - early_hit_ranked_df).abs().sort_values()

### What if we limit the rank calculations to Annual resolution, to account for the fact that reporting isn't even?

In [ ]:
nodate_early_hit_ranked_df = ranked_df.drop(
    ['others', 'non_who'], axis=1
)
nodate_early_hit_ranked_df = nodate_early_hit_ranked_df.applymap(lambda x: pd.to_datetime(x.strftime('2020-01-01') if not pd.isnull(x) else pd.NaT))
nodate_early_hit_ranked_df.head()

In [ ]:
nodate_early_hit_ranked_df = ehr.calculate_combined_rank(nodate_early_hit_ranked_df.transpose())
nodate_early_hit_ranked_df.name = 'nodate_early_hit_ranked_df'

In [ ]:
bob = foo[[
    'outbreak_rank_global', 'new_deaths_smoothed_per_million', 'reproduction_rate', 'new_tests_per_thousand', 'total_vaccinations_per_hundred',
    'population_density', 'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate', 'diabetes_prevalence', 
    'female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand', 'life_expectancy', 'human_development_index', 'population'
]].merge(nodate_early_hit_ranked_df, on='location', how='left')
rho = bob.corr()
num_comparisons = pow(len(rho.columns),2) - len(rho.columns)
pval = bob.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=(t/num_comparisons)]))
rho.round(2).astype(str) + p

fig, ax = plt.subplots(figsize=(12, 12))
im = ax.imshow(rho)

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(rho.columns)), labels=rho.columns)
ax.set_yticks(np.arange(len(rho.columns)), labels=rho.columns)
# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i, iname in enumerate(rho.columns):
    for j, jname in enumerate(rho.columns):
        text = ax.text(j, i, p.loc[iname, jname],
                       ha="center", va="center", color="w", size=14)
   
plt.colorbar(im, ax=ax)
plt.title("Correlation between per variant outbreak order and various country indicators\nNote: lower outbreak_rank_global indicates a country was hit earlier")
plt.show()

In [ ]:
pearsonr(nodate_early_hit_ranked_df, early_hit_ranked_df)

In [ ]:
plt.scatter(nodate_early_hit_ranked_df, early_hit_ranked_df)
plt.ylabel('early_hit_ranked')
plt.xlabel('nodate_early_hit_ranked_df')